# The Big Picture - Context Retrieval

Notebook to combine high-resolution FMoW satellite image with broader but lower-resolution Landsat image from Google Earth Engine by patching the high-res image into the low-res image.

### Requirements
The following python packages must be installed:

```
- boto3 (for AWS S3 access)
- earthengine-api (for Google Earth Engine access)
- numpy
- Pillow
- rasterio
- requests
```

In [ ]:
from datetime import datetime, timedelta
import io
import json
import os
import re

import boto3
from botocore import UNSIGNED
from botocore.config import Config
import ee
import numpy as np
from PIL import Image, ImageDraw
import rasterio
import requests
import zipfile

In order to follow the notebook, your data folder structure must align with the functions below and you have to download and extract the metadata `groundtruth.tar.bz2` from fMoW (important for raw coordinates). Either adapt your data folders or the functions accordingly.  
**Download groundtruth:**
```bash
aws s3 cp --no-sign-request s3://spacenet-dataset/Hosted-Datasets/fmow/fmow-rgb/groundtruth.tar.bz2 .  
```
**Recommendation:**
```
├── data
│   ├── groundtruth (extracted from groundtruth.tar.bz2)
│   │   ├── seq_gt
│   │   ├── seq_gt_mapping.json
│   │   ├── test_gt
│   │   ├── test_gt_mapping.json
│   │   ├── train
│   │   └── val
│   └── train (created when downloading fMoW below)
│       └── airport
|       ... 
├── README.md
└── src
    └── playground
        └── the_big_picture.ipynb
```

In [ ]:
PROJECT_ROOT = '/'.join(os.getcwd().split('/')[:-2])
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
METADATA_DIR = os.path.join(DATA_DIR, "groundtruth")

if not (os.path.exists(PROJECT_ROOT) and os.path.exists(DATA_DIR) and os.path.exists(METADATA_DIR)):
    raise NotADirectoryError()

Also to complete patching the high resolution image onto a low resolution picture, you need access to Google Earth Engine API.
See [Cloud project setup](https://developers.google.com/earth-engine/cloud/earthengine_cloud_project_setup#get-access-to-earth-engine) to register a Google Cloud Project and enable the Earth Engine API (can be done without charge).

In [ ]:
# Adjust this to your Google Cloud Project name.
EE_PROJECT_NAME = 'seeing-the-big-picture'

In [ ]:
RGB_PREFIX = f'Hosted-Datasets/fmow/fmow-rgb/'
BUCKET_NAME = 'spacenet-dataset'

S3 = boto3.client(
    's3',
    region_name='us-east-1',
    config=Config(signature_version=UNSIGNED)
)

### fMoW - S3 Access via boto3

In [ ]:
def display_bucket_content():
    response = S3.list_objects_v2(Bucket=BUCKET_NAME, Prefix=RGB_PREFIX)
    object_keys = list(map(lambda dict: dict['Key'].replace("Hosted-Datasets/fmow/fmow-rgb/", ""), response['Contents']))
    for key in object_keys:
        if key.startswith("seq"):
            continue
        print(key)

display_bucket_content()

CHANGELOG.md
IARPA-fMoW.pdf
LICENSE
README.md
fMoW-rgb_seqandgt_v1.2.1.torrent
fMoW-rgb_test_v1.0.0.torrent
fMoW-rgb_trainval_v1.0.0.torrent
fMoW-rgb_val_sample_v1.1.0.torrent
groundtruth.tar.bz2
manifest.json.bz2


### Download fMoW Image - Load Fitting Metadata

In [ ]:
def get_image_paths(image_key):
    """
    Return image path to download to and metadata path to extract
    metadata from.
    """
    folder_structure = '/'.join(os.path.dirname(image_key).split('/')[3:])
    image_dir = f"{DATA_DIR}/{folder_structure}"
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
    metadata_dir = f"{METADATA_DIR}/{folder_structure}"
    image_name = os.path.splitext(os.path.basename(image_key))[0]
    return f"{image_dir}/{image_name}.jpg", f"{metadata_dir}/{image_name}.json"

def download_fmow_image(split, category, sample=0, index=0):
    """
    Download image from fMoW dataset and load metadata from groundtruth
    metadata containing raw coordinates.

    Caution: groundtruth.tar.bz2 must be downloaded and unpacked first in the data folder.
    """
    
    prefix = f'{RGB_PREFIX}{split}/{category}/{category}_{sample}'
    response = S3.list_objects_v2(Bucket=BUCKET_NAME, Prefix=prefix, MaxKeys=100)
    image_files = [obj['Key'] for obj in response['Contents'] 
        if obj['Key'].endswith(('.jpg', '.png', '.tif'))]
    image_key = image_files[min(index, len(image_files)-1)]
    image_path, metadata_path = get_image_paths(image_key)

    print("Downloading image")
    S3.download_file(BUCKET_NAME, image_key, image_path)
    print(f"Downloaded successfully to {image_path}")

    metadata = {}
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)

    return image_path, metadata

image_path, metadata = download_fmow_image("train", "airport", 230, 2)

KeyboardInterrupt: 

### Explore Metadata - Draw Bounding Box

In [ ]:
def draw_circle(draw, x, y, radius=10):
    draw.ellipse((x - radius, y - radius, x + radius, y + radius), fill = 'blue', outline ='blue')

def draw_boxes_and_center(image_path, metadata):
    """
    Explore metadata: Draw bounding box from metadata as well as center of the bounding box and center of the image.
    """
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)

    box_info = metadata.get('bounding_boxes')[0]
    box = box_info.get('box')
    x, y, w, h = box
    rect = [x, y, x + w, y + h]
    draw.rectangle(rect, outline='red', width=10)
    draw_circle(draw, x + w / 2, y + h / 2)
    
    img_center = (metadata.get('img_width') / 2, metadata.get('img_height') / 2)
    draw_circle(draw, *img_center)

    img.show()
    
draw_boxes_and_center(image_path, metadata)

### Compute Image Center Coordinates
Leverage the raw coordinates of the bounding box to compute the coordinates of the image center as a linear combination of the box corrdiantes.

For this approximation the earth is considered flat.

In [ ]:
def compute_image_center_coordinates(metadata):
    
    box_info = metadata.get('bounding_boxes')[0]
    box = box_info.get('box')
    x, y, w, h = box

    img_width = metadata.get('img_width')
    img_height = metadata.get('img_height')
    center = (img_width / 2, img_height / 2)
    top_left_to_center = ((center[0] - x) / w, (center[1] - y) / h)

    raw_location = metadata['bounding_boxes'][0]['raw_location']

    points = re.findall(r"[-\d.]+ [-\d.]+", raw_location)
    coords = [tuple(map(float, p.split())) for p in points]
    top_left_lon, top_left_lat = coords[0]

    box_min_lon = min(c[0] for c in coords)
    box_max_lon = max(c[0] for c in coords)
    box_min_lat = min(c[1] for c in coords)
    box_max_lat = max(c[1] for c in coords)

    box_width_deg = box_max_lon - box_min_lon
    box_height_deg = box_max_lat - box_min_lat

    center_lon = top_left_lon + top_left_to_center[0] * box_width_deg
    center_lat = top_left_lat - top_left_to_center[1] * box_height_deg

    img_width_deg = img_width / w * box_width_deg
    img_height_deg = img_height / h * box_height_deg

    return (center_lon, center_lat), (img_width_deg, img_height_deg)

center, img_deg = compute_image_center_coordinates(metadata)
print(f"Image center coords: {center}")
print(f"Image span in degree: {img_deg}")

Image center coords: (-17.03336283914791, 20.929515596799998)
Image span in degree: (0.045015799309577414, 0.04219890500000313)


### Download Matching Landsat Image
The computed coordinates of the high-res image together with the high-res image span in degrees are taken to compute a larger geographical region with the high-res image in its center. A `buffer_factor` determines the extend of the bigger box. 

Google Earth Engine can then be requested to search for a Landsat 8 image covering that region.

Scaling and offset factor were taken from [USGS](https://www.usgs.gov/faqs/how-do-i-use-a-scale-factor-landsat-level-2-science-products) and normalization was done according to [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#colab-python).

In [ ]:
def sanitize(value, digits=4):
    """Sanitize values to be included into filenames."""
    s = f"{value:.{digits}f}"
    s = s.replace('.', '_').replace('-', 'neg')
    return s

def normalize_band(band, lower=0, upper=0.3):
    band = np.clip(band, lower, upper)
    band_norm = (band - lower) / (upper - lower)
    return (band_norm * 255).astype(np.uint8)

def download_aligned_lowres_image(image_dir, center, img_deg, buffer_factor=2.0):
    try:
        ee.Authenticate()
        ee.Initialize(project=EE_PROJECT_NAME)
    except Exception as e:
        print("Please authenticate Earth Engine: earthengine authenticate")
        raise
    
    center_lon, center_lat = center
    img_width_deg, img_height_deg = img_deg
    half_width = (img_width_deg * buffer_factor) / 2
    half_height = (img_height_deg * buffer_factor) / 2
    bbox = [center_lon - half_width, center_lat - half_height, center_lon + half_width, center_lat + half_height]
    region = ee.Geometry.Rectangle(bbox)

    # Landsat 8 Surface Reflectance collection
    collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                  .filterBounds(region)
                  .sort('CLOUD_COVER'))

    # Least cloudy image gets selected
    image = collection.first()

    # Red, green and blue bands are selected.
    # Scale and offset factor are applied. 
    rgb = image.select(['SR_B.']).multiply(0.0000275).add(-0.2).clip(region)

    outfiles = {}
    for band, name in zip(['SR_B4', 'SR_B3', 'SR_B2'], ['red.tif', 'green.tif', 'blue.tif']):
        url = rgb.select(band).getDownloadURL({
            'scale': 30,
            'crs': 'EPSG:4326',
            'region': region.bounds().getInfo()['coordinates'],
            'fileFormat': 'GeoTIFF'
        })
        r = requests.get(url)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall(image_dir)
        tiff_filename = [f for f in z.namelist() if f.endswith('.tif')][0]
        tiff_path = os.path.join(image_dir, tiff_filename)
        outfiles[name] = tiff_path

    with rasterio.open(outfiles['red.tif']) as rsrc, \
         rasterio.open(outfiles['green.tif']) as gsrc, \
         rasterio.open(outfiles['blue.tif']) as bsrc:

        red = rsrc.read(1).astype(float)
        green = gsrc.read(1).astype(float)
        blue = bsrc.read(1).astype(float)

        red_n = normalize_band(red)
        green_n = normalize_band(green)
        blue_n = normalize_band(blue)

        rgb_array = np.dstack((red_n, green_n, blue_n))
        rgb_img = Image.fromarray(rgb_array)

        rgb_img_name = (
            f"landsat8_center_"
            f"{sanitize(center_lon)}_{sanitize(center_lat)}"
            f"_size_{sanitize(img_width_deg)}_{sanitize(img_height_deg)}"
            f"_buffer_{sanitize(buffer_factor, 1)}.jpg"
        )
             
        rgb_img_path = os.path.join(image_dir, rgb_img_name)
        rgb_img.save(rgb_img_path)

    for tiff_path in outfiles.values():
        os.remove(tiff_path)
    return rgb_img_path

buffer_factor = 3.0
low_res_img_path = download_aligned_lowres_image(os.path.dirname(image_path), center, img_deg, buffer_factor)

### Composition
In order to patch the Landsat image onto the fMoW sample, the prior needs to be upscaled according to the buffer factor.

In [ ]:
def compose_high_low_res(high_res_img_path, low_res_img_path, buffer_factor):
    """
    Compose the high-res image with the low-res image. 
    Scale the low-res image up, so that it matches the resolution of the high-res image according to its buffer_factor
    and patch the high-res image onto the center of the upscaled image eventually.
    """
    
    high_res = Image.open(image_path)
    high_res_w, high_res_h = high_res.size
    upscaled_size = (int(high_res_w * buffer_factor), int(high_res_h * buffer_factor))
    
    low_res = Image.open(low_res_img_path)
    upscaled_low_res = low_res.resize(upscaled_size, Image.LANCZOS)
    
    composite = upscaled_low_res.copy()
    x0 = (upscaled_size[0] - high_res_w) // 2
    y0 = (upscaled_size[1] - high_res_h) // 2
    composite.paste(high_res, (x0, y0))
    
    dir_name = os.path.dirname(high_res_img_path)
    high_res_name = os.path.splitext(os.path.basename(high_res_img_path))[0]
    low_res_name = os.path.splitext(os.path.basename(low_res_img_path))[0]   
    composite_name = f"compose_{high_res_name}_{low_res_name}.jpg"
    composite.save(os.path.join(dir_name, composite_name))
    
    composite.show()

compose_high_low_res(image_path, low_res_img_path, buffer_factor)

### Further?
- What to do with fMoW samples, where Landsat8 is not available? (e.g. `airport 18`, `barn 4`)
- What about smaller objects like `barn`. Does patching them with Landsat brings too much resolution difference with it?
- How to proceed with color differences?